In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from scipy import interpolate
from scipy.optimize import curve_fit

In [ ]:
data = pd.read_table("g1a.snap_092_dm_particles.klaus",sep='\s+',skiprows=1,
                     header=None,names=['x','y','z','vx','vy','vz'])

In [ ]:
data.head()

In [ ]:
data.shape

# 3D density of dm-particles

In [ ]:
N_per_bin=10000
indices = np.arange(0,data.shape[0],N_per_bin)

In [ ]:
data['r2d'] = np.sqrt(data['x']**2+data['y']**2)
data['r3d'] = np.sqrt(data['x']**2+data['y']**2+data['z']**2)

data.sort_values(by=['r3d'], ascending=True, inplace=True)
data=data.reset_index()

In [ ]:
rbins3d=np.array([data.r3d[i] for i in indices])

In [ ]:
rc3d = (rbins3d[:-1]+rbins3d[1:])/2.
volume = 4./3.*np.pi*(rbins3d[1:]**3-rbins3d[:-1]**3)

In [ ]:
rho3d = N_per_bin/volume

In [ ]:
plt.loglog(rc3d,rho3d,'.')

In [ ]:
def NFW(x, rho0, rs):
    _x = x/rs
    return rho0/_x/(1.+_x)**2

In [ ]:
NFWfit, pcov = curve_fit(NFW, rc3d, rho3d)

In [ ]:
NFWfit

In [ ]:
plt.loglog(rc3d, NFW(rc3d, *NFWfit), 'r-')

plt.loglog(rc3d,rho3d,'.')
plt.xlabel("kpc")
plt.ylabel("density, arb units")

In [ ]:
mass_dm =  0.16156486 # 10^{10}solar masses

In [ ]:
m_unit = 1.9890000000000001e+43
l_unit = 3.0856779999999998e+21


In [ ]:
# M(r)
mass = N_per_bin*mass_dm*m_unit*np.arange(1,len(rbins3d))
av_rho = mass/(4./3. *np.pi*rbins3d[1:]**3*l_unit**3)

In [ ]:
N_per_bin*(len(rbins3d)-1)

In [ ]:
data[data.r3d<np.max(rbins3d)].shape

# $\rho_{crit} = \frac{3H^2}{8\pi G}$

In [ ]:
h0=0.7
rho_crit=3.*(h0*100.*1e5/3.08568e+24)**2/(8.*np.pi*6.67259e-08) #g/cm^3
print(rho_crit, "g/cm^3")

In [ ]:
plt.loglog(rbins3d[1:], av_rho, 'r-')
plt.loglog(rbins3d[1:],rbins3d[1:]*0.+200.*rho_crit,'-')
plt.xlabel("kpc")
plt.ylabel("average density, g/cm^3")


In [ ]:
idx = np.argwhere(np.diff(np.sign(av_rho-200.*rho_crit))).flatten()
print(rbins3d[idx+1])
#plt.loglog([rbins3d[idx+1],rbins3d[idx+1]],[1e-28,1e-23],'--',color='black')


In [ ]:
r200 = rbins3d[idx+1]
M200 = mass[idx]/1.9890000000000001e+33
print(r200,M200)

In [ ]:
plt.hist(data['vz'],bins=100)

In [ ]:
data['vz'].mean()

In [ ]:
data['vz'].std()

In [ ]:
G = 4.302e-3 #pc/Msun (km/s)^2

In [ ]:
Mass=r200*3.*data['vz'].std()*data['vz'].std()/G/1e-3/1e15
print(Mass, "10^{15} Msun")

# surface brightness

In [ ]:
data.sort_values(by=['r2d'], ascending=True, inplace=True)
data=data.reset_index()

In [ ]:
data.head()

In [ ]:
N_per_bin=10000

indices = np.arange(0,data.shape[0],N_per_bin)


In [ ]:
rbins=np.array([data.r2d[i] for i in indices])

In [ ]:
rc = (rbins[:-1]+rbins[1:])/2.
area = np.pi*(rbins[1:]**2-rbins[:-1]**2)

In [ ]:
SB = N_per_bin/area

In [ ]:
plt.loglog(rc,SB,'.')
plt.xlabel("projected dist., kpc")
plt.ylabel("mass/area, arb units")
plt.title("Surface brightness profile")

In [ ]:
Ntotal = N_per_bin*(len(rbins)-1)
idx = np.argwhere(np.sign(N_per_bin*np.arange(1,len(rbins))-Ntotal/2)==0).flatten()
print(idx)
print(rbins[idx])


In [ ]:

#idx=np.argwhere(np.diff(np.sign(N_per_bin*np.arange(1,len(rbins))-Ntotal/2))).flatten()
#idx

In [ ]:
r_halfmass = rbins[idx]


# $r_{halfmass} \simeq 0.45 r_{cluster}$

In [ ]:
Mass=r_halfmass/0.45*3.*data['vz'].std()*data['vz'].std()/G/1e-3/1e15
print(Mass, "10^{15} Msun")

# Неплохо!